## 내 집 마련을 위해, 서울 아파트 매물 데이터를 이용하여 구매 전략을 세우고자 한다.

real_estate.csv

|컬럼|정의|type|
|:---|:---|:---:|
|latitude|위도|float64|
|longitude|경도|float64|
|price_per_square_py|평당가|float64|
|py|평수|int64|
|apt_code|시공사 코드(길이5 알파벳 대문자)|object|
|dist_from_station|인근 지하철역과의 거리|float64|  

정답 및 해설 : https://tjd229.tistory.com/22

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("../Dataset/real_estate.csv")
df

,latitude,longitude,price_per_square_py,py,apt_code,dist_from_station
0,37.125541,126.913776,2860.053787,24,GEDAE,1588.406226
1,37.223169,127.026913,1715.073163,59,IHEAD,986.146800
2,37.239029,126.906303,1780.604630,57,FHIBJ,1492.514512
3,37.274433,126.737771,1508.814336,42,JBCIJ,1621.602866
4,37.082670,127.143634,2933.930488,32,EDJCI,945.034519
...,...,...,...,...,...,...
772,37.161852,126.917406,3564.856667,40,IHEAD,1593.663253
773,37.288945,127.118062,3072.810919,24,GBDGG,1803.257080
774,37.234408,126.745186,2491.714188,61,EJAIA,1346.821965
775,37.294494,127.185185,1746.372819,40,EGFBJ,1461.672552


### Q1. 시공사별 아파트 평수의 평균을 구하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 데이터의 첫 번째 행부터 열 번째 행까지의 시공사 코드(apt_code) 리스트를 저장한다. 첫 번째 행의 데이터는 위도, 경도, 평당가, 평수, 시공사 코드, 인근 지하철역과의 거리가 각각 37.125541, 126.913776, 2860.053787, 24, GEDAE, 1588.406226이다.  
단계 2 : 시공사 코드(apt_code)가 단계 1에서 구한 리스트에 포함되어 있는 데이터를 전부 제거한다.  
단계 3 : 시공사 코드(apt_code)별 평수 평균을 구하고 그 중 가장 큰 값을 구하시오  

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [17]:
df1 = df.copy()

aptlist = df1.loc[:10,'apt_code'].to_list()
print(df1.shape)
print(aptlist)

df1_1 = df1.loc[~df1['apt_code'].isin(aptlist)].copy()

g1 = df1_1.groupby('apt_code')['py'].mean().sort_values(ascending=False)
round(g1[0],2)
#
# print(df1_1.shape)

(777, 6)
['GEDAE', 'IHEAD', 'FHIBJ', 'JBCIJ', 'EDJCI', 'EHCAD', 'HEBIA', 'HEBIA', 'IIGFI', 'FHIBJ', 'GBDGG']


/tmp/ipykernel_67269/3095724469.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  round(g1[0],2)


38.63

### Q2. 아파트 매물을 분석하기 위해 군집화를 하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~9개 중 K-means Silhouette 를 통해 구하고, 이 중 첫번째로 높은 score를 최적의 K로 설정한다.  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)를 제외한 모든 컬럼  

단계 2 : 단계 1에서 최적의 K로 도출한 각 군집의 비율 중, 가장 큰 값을 구하시오  

※ 정답은 올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [37]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

df2 = df.copy()
df2_1 = df2.drop('apt_code', axis=1)

res = []
for k in range(2,10):
  model = KMeans(n_clusters= k, random_state=229)
  model.fit(df2_1)
  score = silhouette_score(df2_1, model.labels_)
  res.append((score, k))

res = sorted(res, reverse=True)
print(res)
K = res[0][1]
print(f"K={K}")

model = KMeans(n_clusters= K, random_state=229)
model.fit(df2_1)
cls1 = model.labels_.sum()
cls1
cls2 = len(df2_1) - cls1
print(np.ceil(cls2 / len(df2_1)*100) /100)
# df2_1['index'] = model.fit(df2_1)
# round(df2_1['index'].value_counts(normalize=True).reset_index(drop=True)[0],2)

  

/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 1

[(0.6346359247298529, 2), (0.4538787546673407, 4), (0.4488577522242527, 3), (0.4373736588661463, 5), (0.3966923382037754, 9), (0.38519356885972655, 7), (0.3778948124317996, 8), (0.35894077704895794, 6)]
K=2
0.75


/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score    
random_state = 229  
문제 지시 외 Default 값 사용  

### Q3. 아파트 가격을 예측하는 모델을 만드려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 평당가(price_per_square_py)와 평수(py)를 곱하여 실제 가격 변수를 생성한다.  
단계 2 : Train Set과 Test Set을 28/30:2/30 비율로 나눈다.  
단계 3 : Train Set으로 LinearRegression 모델을 학습하고, Test Set에 적용한다. 이 때, 절편을 포함한 경우와 포함하지 않은 경우 각각에 대하여 수행한다.  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)와 총 가격을 제외한 모든 컬럼  
            - 종속 변수 : 실제 가격  

단계 3에서 학습한 각 모델에 대하여 예측 결과를 아래 정의된 Measure M으로 계산하였을 때 가장 큰 값은?  
$$ M = \biggl(\frac{1}{n} \sum_{i=1}^{n}{(y_i - \hat{y_i})^2}\biggr), \quad \hat{y_i}: 예측값, y_i: 실제값 $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df3 = df.copy()

df3['total_price'] = df3['price_per_square_py'] * df3['py']


trin_st, test_st= train_test_split(df3, test_size=0.0666, random_state=229)

model = LinearRegression()
# model.fit(X_train, y_train)
model.fit(X=trin_st.drop(columns=['total_price','apt_code']), y = trin_st['total_price'])

pred = model.predict(X = test_st.drop(columns=['total_price','apt_code']))
true = test_st['total_price']
res = (((pred - true)**2).mean())


#LinearRegression.fit(X_train, y_train, intercept=True)

In [50]:
model = LinearRegression(fit_intercept=False)
model.fit(X=trin_st.drop(columns=['total_price','apt_code']), y = trin_st['total_price'])
pred = model.predict(X = test_st.drop(columns=['total_price','apt_code']))
true = test_st['total_price']
res2 = (((pred -true)**2).mean())


round(max(res,res2),2)

19007465.67

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split  
모든 random_state = 229  
문제 지시 외 Default 값 사용  